In [ ]:
N = 1

In [ ]:
# Setup for binSIDH

import sys
import random
import time
import pandas as pd

from montgomery_xz import KummerLine
from montgomery_isogeny_original import KummerLineIsogenyComposite
from utilities import torsion_basis

proof.all(False)
random.seed(int(42))

lam = 128
ternary = False

for arg in sys.argv[1:]:
    if arg.lower() in ["--192"]:
        lam = 192
        break
    elif arg.lower() in ["--256"]:
        lam = 256
        break

for arg in sys.argv[1:]:
    if arg.lower() in ["--bin", "--binary"]:
        ternary = False
        break

params_binSIDH = [134, 192, 256]
params_terSIDH = [93, 128, 162]

if ternary:
    params = params_terSIDH
    keytypes = [[0, 1, 2], [0], [1], [2], [0, 1], [0, 2], [1, 2]]
else:
    params = params_binSIDH
    keytypes = [[1, 2], [1], [2]]

if lam == 128:
    t = params[0]
elif lam == 192:
    t = params[1]
elif lam == 256:
    t = params[2]
else:
    raise Exception("The security parameter needs to be 128, 192, or 256.")


def make_prime(p):
    '''
    Given a value `p`, finds a cofactor `f`
    such that p*f - 1 is prime.
    '''
    for i in range(1000):
        if (p*i - 1).is_prime():
            return p*i - 1, i

P = Primes()[:2*t:2]
Q = Primes()[1:2*t:2]


def compute_kernel_scalars(s, Alice=True):
    """ 
    Given a ternary secret `s`, returns scalars `B1` and `B2`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B1]*P + [B2]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B1]*P and [B2]*Q, which is used in the isogeny computations.
    """
    B1 = 1
    B2 = 1
    order1 = 1
    order2 = 1

    t = len(s)

    if Alice:
        Prime = P
    else:
        Prime = Q
    
    for i, p in enumerate(Prime):
        if Alice and i == 0:
            p = 4

        if s[i] == 2:
            B2 *= p
            order1 *= p
        elif s[i] == 1:
            B1 *= p
            order2 *= p
        else:
            B1 *= p
            B2 *= p
    
    return B1, B2, order1, order2


##### Setup ############################

A = 2*prod(Primes()[:2*t:2]) # The 2* ensures that p \equiv 3 mod 4
B = prod(Primes()[1:2*t:2])

p, f = make_prime(A*B)


FF.<x> = GF(p)[]
F.<i> = GF(p^2, modulus=x^2+1) 
E0 = EllipticCurve(F, [0, 6, 0, 1, 0])
E0.set_order((p+1)**2) 

PA, QA = torsion_basis(E0, A)
PB, QB = torsion_basis(E0, B)

## Ensures that 2*PA != (0,0) and
## 2*QA != (0,0), which causes problems in
## computing isogenies with x-only arithmetic
if A//2 * PA == E0(0, 0):
    PA = PA + QA
elif A//2 * QA == E0(0, 0):
    QA = PA + QA 

assert A//2 * PA != E0(0, 0) and A//2 * QA != E0(0, 0)


E0 = KummerLine(E0)
xPA, xQA = E0(PA[0]), E0(QA[0])
xPB, xQB = E0(PB[0]), E0(QB[0])

In [ ]:
# original functions

def compute_kernel_scalars(s, Alice=True):
    """ 
    Given a ternary secret `s`, returns scalars `B1` and `B2`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B1]*P + [B2]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B1]*P and [B2]*Q, which is used in the isogeny computations.
    """
    B1 = 1
    B2 = 1
    order1 = 1
    order2 = 1

    t = len(s)

    if Alice:
        Prime = P
    else:
        Prime = Q

    for i, p in enumerate(Prime):
        if Alice and i == 0:
            p = 4

        if s[i] == 2:
            B2 *= p
            order1 *= p
        elif s[i] == 1:
            B1 *= p
            order2 *= p
        else:
            B1 *= p
            B2 *= p

    return B1, B2, order1, order2


def keygen(A_points, B_points, keylist, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    # to check biased secret key, sk starts with [1, 2, 0]
    sk = [1, 2, 0]
    sk += random.choices(keylist, k=t-3)
    B1, B2, order1, order2 = compute_kernel_scalars(sk, Alice=Alice)
    sk = (B1, B2, order1, order2)

    # Compute the isogeny kernels
    xK1 = xP * B1
    xK2 = xQ * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E0, xK1, order1)
    xK2 = phi1(xK2)

    # Evaluate action on aux data
    xR, xS = phi1(xR), phi1(xS)

    # Compute the second isogeny from the codomain of phi0
    E1 = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E1, xK2, order2)

    # Evaluate action on aux data
    xR, xS = phi2(xR), phi2(xS)

    # Generate the masking values
    modulus = B if Alice else A  # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E2 = phi2.codomain()
    pk = (E2, xR, xS)

    return sk, pk


def shared(sk, pk):
    # Parse the private/public keys
    B1, B2, order1, order2 = sk
    E, xR, xS = pk

    # Compute the isogeny kernels
    xK1 = xR * B1
    xK2 = xS * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E, xK1, order1)
    xK2 = phi1(xK2)

    # Compute the second isogeny from the codomain of phi0
    E = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E, xK2, order2)

    EAB = phi2.codomain()
    return EAB.j_invariant()

In [ ]:
# implementation of original binSIDH

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from montgomery_isogeny_original import KummerLineIsogenyComposite

keygen_timings_binSIDH = []

for keylist in keytypes:
    tt = [0, 0, 0, 0]
    ttr = []

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {(tt[0]):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_binSIDH.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'1,2': [keygen_timings_binSIDH[0]],
     '1': [keygen_timings_binSIDH[1]],
     '2': [keygen_timings_binSIDH[2]],
     },
    index=['timing']
)

In [ ]:
# Setup for terSIDH

import sys
import random
import time
import pandas as pd

from montgomery_xz import KummerLine
from montgomery_isogeny_original import KummerLineIsogenyComposite
from utilities import torsion_basis


proof.all(False)
random.seed(int(42))

lam = 128
ternary = True

for arg in sys.argv[1:]:
    if arg.lower() in ["--192"]:
        lam = 192
        break
    elif arg.lower() in ["--256"]:
        lam = 256
        break

for arg in sys.argv[1:]:
    if arg.lower() in ["--bin", "--binary"]:
        ternary = False
        break

params_binSIDH = [134, 192, 256]
params_terSIDH = [93, 128, 162]

if ternary:
    params = params_terSIDH
    keytypes = [[0, 1, 2], [0], [1], [2], [0, 1], [0, 2], [1, 2]]
else:
    params = params_binSIDH
    keytypes = [[1, 2], [1], [2]]

if lam == 128:
    t = params[0]
elif lam == 192:
    t = params[1]
elif lam == 256:
    t = params[2]
else:
    raise Exception("The security parameter needs to be 128, 192, or 256.")


def make_prime(p):
    '''
    Given a value `p`, finds a cofactor `f`
    such that p*f - 1 is prime.
    '''
    for i in range(1000):
        if (p*i - 1).is_prime():
            return p*i - 1, i

P = Primes()[:2*t:2]
Q = Primes()[1:2*t:2]


def compute_kernel_scalars(s, Alice=True):
    """ 
    Given a ternary secret `s`, returns scalars `B1` and `B2`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B1]*P + [B2]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B1]*P and [B2]*Q, which is used in the isogeny computations.
    """
    B1 = 1
    B2 = 1
    order1 = 1
    order2 = 1

    t = len(s)

    if Alice:
        Prime = P
    else:
        Prime = Q
    
    for i, p in enumerate(Prime):
        if Alice and i == 0:
            p = 4

        if s[i] == 2:
            B2 *= p
            order1 *= p
        elif s[i] == 1:
            B1 *= p
            order2 *= p
        else:
            B1 *= p
            B2 *= p
    
    return B1, B2, order1, order2


##### Setup ############################

A = 2*prod(Primes()[:2*t:2]) # The 2* ensures that p \equiv 3 mod 4
B = prod(Primes()[1:2*t:2])

p, f = make_prime(A*B)


FF.<x> = GF(p)[]
F.<i> = GF(p^2, modulus=x^2+1) 
E0 = EllipticCurve(F, [0, 6, 0, 1, 0])
E0.set_order((p+1)**2) 

PA, QA = torsion_basis(E0, A)
PB, QB = torsion_basis(E0, B)

## Ensures that 2*PA != (0,0) and
## 2*QA != (0,0), which causes problems in
## computing isogenies with x-only arithmetic
if A//2 * PA == E0(0, 0):
    PA = PA + QA
elif A//2 * QA == E0(0, 0):
    QA = PA + QA 

assert A//2 * PA != E0(0, 0) and A//2 * QA != E0(0, 0)


E0 = KummerLine(E0)
xPA, xQA = E0(PA[0]), E0(QA[0])
xPB, xQB = E0(PB[0]), E0(QB[0])

In [ ]:
# implementation of original terSIDH

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

keygen_timings_terSIDH = []

for keylist in keytypes:
    tt = [0, 0, 0, 0]
    ttr = []

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {(tt[0]):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_terSIDH.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original': [keygen_timings_terSIDH[0]],
     '0': [keygen_timings_terSIDH[1]],
     '1': [keygen_timings_terSIDH[2]],
     '2': [keygen_timings_terSIDH[3]],
     '0,1': [keygen_timings_terSIDH[4]],
     '0,2': [keygen_timings_terSIDH[5]],
     '1,2': [keygen_timings_terSIDH[6]]},
    index=['timing']
)

In [ ]:
# implementation of reverse terSIDH
from montgomery_isogeny_reverse import KummerLineIsogenyComposite

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

keygen_timings_reverse = []

for keylist in keytypes:
    tt = [0, 0, 0, 0]
    ttr = []

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {(tt[0]):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_reverse.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original': [keygen_timings_reverse[0]],
     '0': [keygen_timings_reverse[1]],
     '1': [keygen_timings_reverse[2]],
     '2': [keygen_timings_reverse[3]],
     '0,1': [keygen_timings_reverse[4]],
     '0,2': [keygen_timings_reverse[5]],
     '1,2': [keygen_timings_reverse[6]]},
    index=['timing']
)

In [ ]:
# reverse fucntion
from montgomery_isogeny_reverse import KummerLineIsogenyComposite

N = 10

def keygen_reverse(A_points, B_points, keylist, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    # to check biased secret key, sk starts with [1, 2, 0]
    sk = [1, 2, 0]
    sk += random.choices(keylist, k=t-3)
    B1, B2, order1, order2 = compute_kernel_scalars(sk, Alice=Alice)
    sk = (B1, B2, order1, order2)

    # Compute the isogeny kernels
    xK1 = xP * B1
    xK2 = xQ * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E0, xK1, order1)
    xK2 = phi1(xK2)

    # Evaluate action on aux data
    xR, xS = phi1(xR), phi1(xS)

    # Compute the second isogeny from the codomain of phi0
    E1 = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E1, xK2, order2)

    # Evaluate action on aux data
    xR, xS = phi2(xR), phi2(xS)

    # Generate the masking values
    modulus = B if Alice else A  # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E2 = phi2.codomain()
    pk = (E2, xR, xS)

    return sk, pk


def shared_reverse(sk, pk):
    # Parse the private/public keys
    B1, B2, order1, order2 = sk
    E, xR, xS = pk

    # Compute the isogeny kernels
    xK1 = xR * B1
    xK2 = xS * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E, xK1, order1)
    xK2 = phi1(xK2)

    # Compute the second isogeny from the codomain of phi0
    E = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E, xK2, order2)

    EAB = phi2.codomain()
    return EAB.j_invariant()

In [ ]:
# implementation of reverse terSIDH
from montgomery_isogeny_reverse import KummerLineIsogenyComposite

keygen_timings_reverse = []

for keylist in keytypes:
    tt = [0, 0, 0, 0]
    ttr = []

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen_reverse((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen_reverse((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared_temp(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared_temp(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {((tt[0]+tt[1])/2):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_reverse.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original': [keygen_timings_reverse[0]]},
    index=['timing']
)

In [ ]:
# random fucntion
from montgomery_isogeny_random import KummerLineIsogenyComposite

def keygen_random(A_points, B_points, keylist, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    # to check biased secret key, sk starts with [1, 2, 0]
    sk = [1, 2, 0]
    sk += random.choices(keylist, k=t-3)
    B1, B2, order1, order2 = compute_kernel_scalars(sk, Alice=Alice)
    sk = (B1, B2, order1, order2)

    # Compute the isogeny kernels
    xK1 = xP * B1
    xK2 = xQ * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E0, xK1, order1)
    xK2 = phi1(xK2)

    # Evaluate action on aux data
    xR, xS = phi1(xR), phi1(xS)

    # Compute the second isogeny from the codomain of phi0
    E1 = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E1, xK2, order2)

    # Evaluate action on aux data
    xR, xS = phi2(xR), phi2(xS)

    # Generate the masking values
    modulus = B if Alice else A  # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E2 = phi2.codomain()
    pk = (E2, xR, xS)

    return sk, pk


def shared_random(sk, pk):
    # Parse the private/public keys
    B1, B2, order1, order2 = sk
    E, xR, xS = pk

    # Compute the isogeny kernels
    xK1 = xR * B1
    xK2 = xS * B2

    # Compute the first isogeny
    phi1 = KummerLineIsogenyComposite(E, xK1, order1)
    xK2 = phi1(xK2)

    # Compute the second isogeny from the codomain of phi0
    E = phi1.codomain()
    phi2 = KummerLineIsogenyComposite(E, xK2, order2)

    EAB = phi2.codomain()
    return EAB.j_invariant()

In [ ]:
# implementation of random terSIDH
from montgomery_isogeny_random import KummerLineIsogenyComposite

keygen_timings_random = []

for keylist in keytypes:
    tt = [0, 0, 0, 0]
    ttr = []

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen_random((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen_random((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared_temp(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared_temp(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {((tt[0]+tt[1])/2):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_random.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original': [keygen_timings_random[0]]},
    index=['timing']
)

In [ ]:
# Setup for constant terSIDH ver.1

from montgomery_isogeny_constant import KummerLineIsogenyComposite


def compute_kernel_scalars_constant(s, Alice=True):
    """ 
    Given a ternary secret `s`, returns scalars `B0` and `B1`
    such that the isogeny associated with `s` and orientation (P, Q)
    has kernel <[B0]*P + [B1]*Q>.
    The function also returns `order0` and `order1`, the orders
    of points [B0]*P and [B1]*Q, which is used in the isogeny computations.
    """
    B0 = 1
    B1 = 1
    B2 = 1
    order0 = 1
    order1 = 1
    order2 = 1
    primes = 1

    t = len(s)

    if Alice:
        Primes = P
    else:
        Primes = Q

    for i, p in enumerate(Primes):
        if Alice and i == 0:
            p = 4

        primes *= p
        if s[i] == 2:
            B1 *= p
            order0 *= p
            B2 *= p
        elif s[i] == 1:
            B0 *= p
            order1 *= p
            B2 *= p
        else:
            B0 *= p
            B1 *= p
            order2 *= p

    return B0, B1, B2, order0, order1, order2, primes


def keygen_constant(A_points, B_points, keylist, Alice=True):
    
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points

    # Generate the secret data
    sk = [1, 2, 0]
    sk += random.choices(keylist, k=t-3)
    B0, B1, B2, order0, order1, order2, primes = compute_kernel_scalars_constant(sk, Alice=Alice)
    sk = (B0, B1, B2, order0, order1, order2, primes)

    # Compute the isogeny kernels
    xK0 = xP
    xK1 = xQ
    xK2 = xQ

    # Compute the isogeny
    phi = KummerLineIsogenyComposite(E0, B0, xK0, order0, B1, xK1, order1, B2, xK2, order2, primes)
    xR, xS = phi(xR), phi(xS)

    # Generate the masking values
    modulus = B if Alice else A  # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E = phi.codomain()
    pk = (E, xR, xS)

    return sk, pk


def shared_constant(sk, pk):
    # Parse the private/public keys
    B0, B1, B2, order0, order1, order2, primes = sk
    E, xR, xS = pk

    # Compute the isogeny kernels
    xK0 = xR
    xK1 = xS
    xK2 = xR

    # Compute the first isogeny
    phi2 = KummerLineIsogenyComposite(
        E, B0, xK0, order0, B1, xK1, order1, B2, xK2, order2, primes)

    EAB = phi2.codomain()
    
    return EAB.j_invariant()
    

In [ ]:
# implementation of constant terSIDH

keygen_timings_constant = []

for keylist in keytypes:

    tt = [0, 0, 0, 0]

    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen_constant((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        skB, pkB = keygen_constant((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssA = shared_temp(skA, pkB)
        tt[2] += time.process_time_ns() - t0

        t0 = time.process_time_ns()
        ssB = shared_temp(skB, pkA)
        tt[3] += time.process_time_ns() - t0

        assert ssA == ssB

    tt = [float(t) / N / 10 ^ 6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {(tt[0]):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_constant.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original': [keygen_timings_constant[0]],
     '0': [keygen_timings_constant[1]],
     '1': [keygen_timings_constant[2]],
     '2': [keygen_timings_constant[3]],
     '0,1': [keygen_timings_constant[4]],
     '0,2': [keygen_timings_constant[5]],
     '1,2': [keygen_timings_constant[6]]},
    index=['timing']
)

In [ ]:
# Setup for terSIDH ver.2
from montgomery_isogeny_ours import KummerLineIsogenyComposite


def keygen_ours(A_points, B_points, keylist, Alice=True):
    # Extract Kummer points
    xP, xQ = A_points
    xR, xS = B_points
    xT = xQ # dummy point
    
    # Generate the secret data
    # to check biased secret key, sk starts with [1, 2, 0]
    sk = [1, 2, 0]
    sk += random.choices(keylist, k=t-3)
    random.shuffle(sk)

    # Compute isogeny kernel
    if Alice:
        Prime = P        
        primes = A
        modulus = B
    else:
        Prime = Q        
        primes = B
        modulus = A

    order1 = 1
    order2 = 1
    order0 = 1

    for i, p in enumerate(Prime):
        if Alice and i == 0:
            p = 4

        if sk[i] == 2:
            xQ *= p
            order1 *= p
            xT *= p
        elif sk[i] == 1:
            xP *= p
            order2 *= p
            xT *= p
        else:
            xP *= p
            xQ *= p
            order0 *= p

    xPa, xQa = xP.curve_point(), xQ.curve_point()
    xK0 = E0((xPa + xQa)[0])

    orders = order1 * order2
    
    # Compute the isogeny
    phi = KummerLineIsogenyComposite(E0, xK0, orders, xT, primes)
    xR, xS = phi(xR), phi(xS)

    # Generate the masking values
    # modulus is the order of torsion points
    mask = modulus
    while gcd(mask, modulus) != 1:
        mask = random.randint(0, modulus)
    mask_inv = 1/mask % modulus

    # Scale the torsion images
    xR = mask * xR
    xS = mask_inv * xS

    # Compute the public data
    E = phi.codomain()
    pk = (E, xR, xS)

    return sk, pk

def shared_ours(sk, pk, Alice=True):
    # Parse the private/public keys
    E, xR, xS = pk
    xT = xS # dummy point

    # Compute isogeny kernel
    if Alice:
        Prime = P        
        primes = A
        modulus = B
    else:
        Prime = Q        
        primes = B
        modulus = A

    order1 = 1
    order2 = 1
    order0 = 1
    
    for i, p in enumerate(Prime):
        if Alice and i == 0:
            p = 4

        if sk[i] == 2:
            xS *= p
            order1 *= p
            xT *= p
        elif sk[i] == 1:
            xR *= p
            order2 *= p
            xT *= p
        else:
            xR *= p
            xS *= p
            order0 *= p

    xRa, xSa = xR.curve_point(), xS.curve_point()
    xK = E((xRa + xSa)[0])

    orders = order1 * order2

    # Compute the first isogeny
    phi2 = KummerLineIsogenyComposite(E, xK, orders, xT, primes)

    EAB = phi2.codomain()

    return EAB.j_invariant()

In [ ]:
# implementation of our algorithm

keygen_timings_ours = []

for keylist in keytypes:    
    tt = [0, 0, 0, 0]
    ttr = []
    
    for _ in range(N):
        t0 = time.process_time_ns()
        skA, pkA = keygen_ours((xPA, xQA), (xPB, xQB), keylist, Alice=True)
        tt[0] += time.process_time_ns() - t0
    
        t0 = time.process_time_ns()
        skB, pkB = keygen_ours((xPB, xQB), (xPA, xQA), keylist, Alice=False)
        tt[1] += time.process_time_ns() - t0
    
        t0 = time.process_time_ns()
        ssA = shared_ours(skA, pkB, Alice=True)
        tt[2] += time.process_time_ns() - t0
    
        t0 = time.process_time_ns()
        ssB = shared_ours(skB, pkA, Alice=False)
        tt[3] += time.process_time_ns() - t0
    
        assert ssA == ssB
    
    tt = [float(t) / N / 10^6 for t in tt]

    print("Key type is : ", keylist)
    print(f"KeyGen_A took {(tt[0]):.1f} ms")
    print(f"KeyGen_B took {(tt[1]):.1f} ms")
    print(f"shared_A took {(tt[2]):.1f} ms")
    print(f"shared_B took {(tt[3]):.1f} ms")
    print("")
    keygen_timings_ours.append((tt[0]+tt[1])/2)

pd.DataFrame(
    {'original' : [keygen_timings_ours[0]],
     '0' : [keygen_timings_ours[1]],
     '1' : [keygen_timings_ours[2]],
     '2' : [keygen_timings_ours[3]],
     '0,1' : [keygen_timings_ours[4]],
     '0,2' : [keygen_timings_ours[5]],
     '1,2' : [keygen_timings_ours[6]]},
     index = ['timing']
)

In [ ]:
# time table for order
empty = '-'
col = ['original', '0', '1', '2', '0,1', '0,2', '1,2']
ind = ['original terSIDH', 'random terSIDH', 'reverse terSIDH']
con = [[keygen_timings_terSIDH[0], keygen_timings_terSIDH[1], keygen_timings_terSIDH[2], 
        keygen_timings_terSIDH[3], keygen_timings_terSIDH[4], keygen_timings_terSIDH[5], 
        keygen_timings_terSIDH[6]],
        [keygen_timings_random[0], keygen_timings_random[1], keygen_timings_random[2],
        keygen_timings_random[3], keygen_timings_random[4], keygen_timings_random[5],
        keygen_timings_random[6]],
       [keygen_timings_reverse[0], keygen_timings_reverse[1], keygen_timings_reverse[2], 
        keygen_timings_reverse[3], keygen_timings_reverse[4], keygen_timings_reverse[5], 
        keygen_timings_reverse[6]]]

df = pd.DataFrame(con, columns=col, index=ind)
df

In [ ]:
# time table
empty = '-'
col = ['original', '0', '1', '2', '0,1', '0,2', '1,2']
ind = ['original binSIDH', 'original terSIDH', 'constant terSIDH', 'ours terSIDH']
con = [[keygen_timings_binSIDH[0], "-", keygen_timings_binSIDH[1], 
        keygen_timings_binSIDH[2], "-", "-", 
        keygen_timings_binSIDH[0]],
       [keygen_timings_terSIDH[0], keygen_timings_terSIDH[1], keygen_timings_terSIDH[2], 
        keygen_timings_terSIDH[3], keygen_timings_terSIDH[4], keygen_timings_terSIDH[5], 
        keygen_timings_terSIDH[6]],
       [keygen_timings_constant[0], keygen_timings_constant[1], keygen_timings_constant[2], 
        keygen_timings_constant[3], keygen_timings_constant[4], keygen_timings_constant[5], 
        keygen_timings_constant[6]],
       [keygen_timings_ours[0], keygen_timings_ours[1], keygen_timings_ours[2],
        keygen_timings_ours[3], keygen_timings_ours[4], keygen_timings_ours[5],
        keygen_timings_ours[6]]]

df = pd.DataFrame(con, columns=col, index=ind)
df